In [1]:
from xml.dom import minidom
import simplejson as json
def parse_element(element):
    dict_data = dict()
    if element.nodeType == element.TEXT_NODE:
        if element.data != " ":
            dict_data['data'] = element.data
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_NODE, 
                                element.DOCUMENT_TYPE_NODE]:
        for item in element.attributes.items():
            dict_data[item[0]] = item[1]
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_TYPE_NODE]:
        for child in element.childNodes:
            child_name, child_dict = parse_element(child)
            if child_name in dict_data:
                try:
                    dict_data[child_name].append(child_dict)
                except AttributeError:
                    dict_data[child_name] = [dict_data[child_name], child_dict]
            else:
                dict_data[child_name] = child_dict 
    return element.nodeName, dict_data

if __name__ == '__main__':
    dom = minidom.parse('cqa_task3.xml')
    print type(dom)
    f = open('cqa_task3.json', 'w')
    f.write(json.dumps(parse_element(dom), sort_keys=True, indent=4))
    f.close()

<type 'instance'>


In [2]:
import pandas as pd

data1 = pd.read_json('cqa_task3.json')
data1.head()
df31 = pd.io.json.json_normalize(data1[0][1]['xml'][1]['Thread'])
df41 = pd.concat([pd.DataFrame(pd.io.json.json_normalize(x)) for x in df31['RelComment']],ignore_index=True)
df41['THREAD_SEQUENCE'] = [ '_'.join(i.split('_')[:2]) for i in df41['RELC_ID']]
df51 = pd.concat([df31.set_index('THREAD_SEQUENCE'),df41.set_index('THREAD_SEQUENCE')], axis=1, join='inner').reset_index()
df51.head()
rem_cols = ['RelQuestion.#text' , 'RelComment', 'RelCBody.#text.data' , 'RelQuestion.RelQBody.#text.data' , 
            'RelQuestion.RelQBody.#text.data', '#text']
[ df51.drop(i, axis=1, inplace=True) for i in rem_cols  ]
df51.head()
#df51[ df51['RelQuestion.RelQSubject.#text.data'] == 'Best Bank.' ]
df51.info()
# good_df = df51[df51['RELC_RELEVANCE2RELQ'] == "Good"]
# print len(good_df)
# bad_df = df51[df51['RELC_RELEVANCE2RELQ'] != "Good"]
# print len(bad_df)
# df51.RELC_RELEVANCE2RELQ.unique()
# len(good_df.RELC_USERID.unique())
# print good_df.columns
# new_df = good_df.groupby([u'RelQuestion.RELQ_USERID'])
# new_df
import seaborn as sns
import matplotlib.pyplot as plt
is_dup = df51['RELC_RELEVANCE2RELQ'].value_counts()
is_dup.head()
plt.figure(figsize=(8,4))
sns.barplot(is_dup.index, is_dup.values, alpha=0.8, color=color[1])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Values', fontsize=12)
plt.show()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2440 entries, 0 to 2439
Data columns (total 14 columns):
THREAD_SEQUENCE                       2440 non-null object
RelQuestion.RELQ_CATEGORY             2440 non-null object
RelQuestion.RELQ_DATE                 2440 non-null object
RelQuestion.RELQ_ID                   2440 non-null object
RelQuestion.RELQ_USERID               2440 non-null object
RelQuestion.RELQ_USERNAME             2440 non-null object
RelQuestion.RelQClean.#text.data      2440 non-null object
RelQuestion.RelQSubject.#text.data    2440 non-null object
RELC_DATE                             2440 non-null object
RELC_ID                               2440 non-null object
RELC_RELEVANCE2RELQ                   2440 non-null object
RELC_USERID                           2440 non-null object
RELC_USERNAME                         2440 non-null object
RelCClean.#text.data                  2440 non-null object
dtypes: object(14)
memory usage: 266.9+ KB


/Users/srimi/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


NameError: name 'color' is not defined

In [ ]:
color = sns.color_palette()
plt.figure(figsize=(8,4))
sns.barplot(is_dup.index, is_dup.values, alpha=0.8, color=color[1])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Values', fontsize=12)
plt.show()

In [3]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, svm
train, test = train_test_split(df51, test_size=0.2)
print len(train),len(test)
col_list =  list(df51.columns)
print len(col_list)
col_list.remove(u'RELC_RELEVANCE2RELQ')
print len(col_list)
train_x = train[col_list]
train_y = train[u'RELC_RELEVANCE2RELQ']
d = train_x.to_dict(orient='records')
train_data = zip(d,train_y)
test_x = test[col_list]
test_y = test[u'RELC_RELEVANCE2RELQ']
d = test.to_dict(orient='records')
test_data = zip(d,test_y)
# classifier = NaiveBayesClassifier.train(train_data)
# print(nltk.classify.accuracy(classifier, test_data))
# print classifier.show_most_informative_features(5)

1952 488
14
13


In [20]:
from nltk.corpus import wordnet as wn
wn.synsets('bank')
# dog.hypernyms()
dog = wn.synset('dog.n.01')
chair = wn.synset('chair.n.01')
dog.path_similarity(chair)

0.08333333333333333

In [4]:
from sklearn.linear_model import Ridge
reg = Ridge(alpha=.1)
reg.fit(train_x, train_y)
predictions = reg.predict(test_x)

ValueError: could not convert string to float: I think after 25 Dec he is busy cleaning his confession booth ;-) [img_assist|nid=58165|title=|desc=|link=none|align=left|width=|height=0]

In [100]:
df_new = df51

In [101]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

def preprocess_words(x):
    example_sent = x
    stop_words = set(stopwords.words('english'))
    tokenizer = RegexpTokenizer(r'\w+')
    word_tokens = tokenizer.tokenize(example_sent)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)


In [102]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

def get_cosine(text1, text2):
     vec1 = text_to_vector(text1)
     vec2 = text_to_vector(text2)
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)
     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator
    
df_new['tokenized_question'] = df_new['RelQuestion.RelQClean.#text.data'].apply(lambda row: preprocess_words(str(''.join([c for c in row if ord(c) < 128]))))
df_new['tokenized_comment'] = df_new['RelCClean.#text.data'].apply(lambda row: preprocess_words(str(''.join([c for c in row if ord(c) < 128]))))
df_new['cosine_similarity_q2c'] = df_new.apply(lambda row : get_cosine( str(row['tokenized_question']),str(row['tokenized_comment']) ),axis =1)
df_new['pos_tag_ques'] = df_new['RelQuestion.RelQClean.#text.data'].apply(lambda x: nltk.pos_tag(x.split()) )
df_new['pos_tag_ans'] = df_new['RelCClean.#text.data'].apply(lambda x: nltk.pos_tag(x.split()) )


In [116]:
df_new.head()
each_df = df_new[ df_new['THREAD_SEQUENCE'] == 'Q268_R16']
#each_df[['RELC_RELEVANCE2RELQ','cosine_similarity_q2c']]
each_df

,THREAD_SEQUENCE,RelQuestion.RELQ_CATEGORY,RelQuestion.RELQ_DATE,RelQuestion.RELQ_ID,RelQuestion.RELQ_USERID,RelQuestion.RELQ_USERNAME,RelQuestion.RelQClean.#text.data,RelQuestion.RelQSubject.#text.data,RELC_DATE,RELC_ID,RELC_RELEVANCE2RELQ,RELC_USERID,RELC_USERNAME,RelCClean.#text.data,tokenized_question,tokenized_comment,cosine_similarity_q2c,pos_tag_ques,pos_tag_ans
0,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-07-31 06:46:39,Q268_R16_C1,Bad,U65,Molten Metal,banks are using us ... Talk to those who had t...,Best Bank Hi ti QL What bank using Are using b...,banks using us Talk taken credit card loan know,0.157135,"[(Best, NNP), (Bank., NNP), (//, NNP), (Hi, NN...","[(banks, NNS), (are, VBP), (using, VBG), (us, ..."
1,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-07-31 08:10:53,Q268_R16_C2,Bad,U956,Rip Cord,In Qatar that is like saying which is the best...,Best Bank Hi ti QL What bank using Are using b...,In Qatar like saying best STD,0.000000,"[(Best, NNP), (Bank., NNP), (//, NNP), (Hi, NN...","[(In, IN), (Qatar, NNP), (that, WDT), (is, VBZ..."
2,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-07-31 10:37:03,Q268_R16_C3,Bad,U5152,arman1arzoo,I'm surprised to see such feedbacks on Qatar b...,Best Bank Hi ti QL What bank using Are using b...,I surprised see feedbacks Qatar banks Is serio...,0.169334,"[(Best, NNP), (Bank., NNP), (//, NNP), (Hi, NN...","[(I'm, NNP), (surprised, VBD), (to, TO), (see,..."
3,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-01 11:38:21,Q268_R16_C4,Good,U5153,westernindoha,Well Arman; nothing is wrong here with banks; ...,Best Bank Hi ti QL What bank using Are using b...,Well Arman nothing wrong banks I feel par UAE ...,0.091574,"[(Best, NNP), (Bank., NNP), (//, NNP), (Hi, NN...","[(Well, RB), (Arman;, NNP), (nothing, NN), (is..."
4,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-01 11:45:49,Q268_R16_C5,Good,U492,happygolucky,With QNB for last 4 years plus...no issues...g...,Best Bank Hi ti QL What bank using Are using b...,With QNB last 4 years plus issues great servic...,0.000000,"[(Best, NNP), (Bank., NNP), (//, NNP), (Hi, NN...","[(With, IN), (QNB, NNP), (for, IN), (last, JJ)..."
5,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-02 06:38:26,Q268_R16_C6,PotentiallyUseful,U5151,shehabi,WesternInDoha; that's the information that I a...,Best Bank Hi ti QL What bank using Are using b...,WesternInDoha information I looking answer que...,0.000000,"[(Best, NNP), (Bank., NNP), (//, NNP), (Hi, NN...","[(WesternInDoha;, NNP), (that's, VBZ), (the, D..."
6,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-02 06:46:37,Q268_R16_C7,PotentiallyUseful,U5151,shehabi,MoltenMetal; it depend how you are looking on ...,Best Bank Hi ti QL What bank using Are using b...,MoltenMetal depend looking subject matter poin...,0.000000,"[(Best, NNP), (Bank., NNP), (//, NNP), (Hi, NN...","[(MoltenMetal;, NN), (it, PRP), (depend, VB), ..."
7,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268_R16,U5151,shehabi,Best Bank. // Hi ti all QL's; What bank you ar...,Best Bank.,2013-08-02 06:51:51,Q268_R16_C8,Bad,U5151,shehabi,That's new way of description for the Bank's.....,Best Bank Hi ti QL What bank using Are using b...,That new way description Bank I planing let co...,0.162221,"[(Best, NNP), (Bank., NNP), (//, NNP), (Hi, NN...","[(That's, NNP), (new, JJ), (way, NN), (of, IN)..."
8,Q268_R16,Moving to Qatar,2013-07-31 02:27:08,Q268